# Deployment

A [Deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/) provides declarative updates for Pods and ReplicaSets.

[Deployment](https://kubernetes.io/docs/concepts/workloads/controllers/deployment/) 為Pods和ReplicaSets提供聲明式的更新能力


In [59]:
from IPython.display import Image
Image(url="./img/deployment.jpg", width=300, height=300)

## 什麼時候用 Deployment

首先，我們一般不會用ReplicaSet，而是直接用Deployment。
因為Deployment有更強大的功能，可以管理ReplicaSet和

## 扩容

首先可以创建一个 deployment.yaml 的文件。来管理 hostname pod。

In [60]:
# ! kubectl explain RESOURCE
! kubectl explain deployment

KIND:     Deployment
VERSION:  apps/v1

DESCRIPTION:
     Deployment enables declarative updates for Pods and ReplicaSets.

FIELDS:
   apiVersion	<string>
     APIVersion defines the versioned schema of this representation of an
     object. Servers should convert recognized schemas to the latest internal
     value, and may reject unrecognized values. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#resources

   kind	<string>
     Kind is a string value representing the REST resource this object
     represents. Servers may infer this from the endpoint the client submits
     requests to. Cannot be updated. In CamelCase. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#types-kinds

   metadata	<Object>
     Standard object's metadata. More info:
     https://git.k8s.io/community/contributors/devel/sig-architecture/api-conventions.md#metadata

   spec	<Object>
     Specification of th

In [61]:
# Get the documentation of a specific field of a resource
! kubectl explain deployment.spec.replicas

KIND:     Deployment
VERSION:  apps/v1

FIELD:    replicas <integer>

DESCRIPTION:
     Number of desired pods. This is a pointer to distinguish between explicit
     zero and not specified. Defaults to 1.


In [62]:
# Get the documentation of a specific field of a resource
! kubectl explain deployment.spec.selector

KIND:     Deployment
VERSION:  apps/v1

RESOURCE: selector <Object>

DESCRIPTION:
     Label selector for pods. Existing ReplicaSets whose pods are selected by
     this will be the ones affected by this deployment. It must match the pod
     template's labels.

     A label selector is a label query over a set of resources. The result of
     matchLabels and matchExpressions are ANDed. An empty label selector matches
     all objects. A null label selector matches no objects.

FIELDS:
   matchExpressions	<[]Object>
     matchExpressions is a list of label selector requirements. The requirements
     are ANDed.

   matchLabels	<map[string]string>
     matchLabels is a map of {key,value} pairs. A single {key,value} in the
     matchLabels map is equivalent to an element of matchExpressions, whose key
     field is "key", the operator is "In", and the values array contains only
     "value". The requirements are ANDed.



In [63]:
# 確保你的minikube有啟動
! [[ -z $(kubectl get nodes) ]] && minikube start 
# 列印目前k8s節點
! kubectl get nodes                    

NAME       STATUS   ROLES           AGE    VERSION
minikube   Ready    control-plane   153m   v1.25.3


In [64]:
# 列出目前 deloyment
! kubectl get deploy

NAME       READY   UP-TO-DATE   AVAILABLE   AGE
hostname   3/3     3            3           37m


In [65]:
# 透過指令建立deployment
! kubectl create deployment hostname --image=brokenpen/hostname:v1 # --ports 8080 
! echo 
! kubectl create deployment hostname --image=brokenpen/hostname:v1 

error: failed to create deployment: deployments.apps "hostname" already exists

error: failed to create deployment: deployments.apps "hostname" already exists


In [66]:
# 列出目前 deloyment
! kubectl get deploy 

NAME       READY   UP-TO-DATE   AVAILABLE   AGE
hostname   3/3     3            3           37m


In [67]:
# 列出目前 deloyment
! kubectl get deploy -o wide

NAME       READY   UP-TO-DATE   AVAILABLE   AGE   CONTAINERS   IMAGES                  SELECTOR
hostname   3/3     3            3           37m   hostname     brokenpen/hostname:v2   app=hostname


In [68]:
# 透過 dryrun, 來預覽yaml檔案，可是並不會實際建立
! kubectl create deployment hostname --image=brokenpen/hostname:v1  --dry-run -o yaml

W1130 10:43:24.166849  539227 helpers.go:663] --dry-run is deprecated and can be replaced with --dry-run=client.
apiVersion: apps/v1
kind: Deployment
metadata:
  creationTimestamp: null
  labels:
    app: hostname
  name: hostname
spec:
  replicas: 1
  selector:
    matchLabels:
      app: hostname
  strategy: {}
  template:
    metadata:
      creationTimestamp: null
      labels:
        app: hostname
    spec:
      containers:
      - image: brokenpen/hostname:v1
        name: hostname
        resources: {}
status: {}


In [69]:
# 使用deployment.yaml 範例建立 deployment
! cat deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hostname
spec:
  replicas: 1
  selector:
    matchLabels:
      app: hostname
  template:
    metadata:
      labels:
        app: hostname
    spec:
      containers:
        - image: docker.io/brokenpen/hostname:v1
          name: hostname
          ports:
            - name: http
              containerPort: 8080
 

- yaml
   - apiVersion
   - kind
   - metadata
   - spec
      - replicas
      - selector
      - template

In [70]:
# 透過yaml檔案建立deployment
! kubectl delete deployment hostname
! echo 
! kubectl apply -f deployment.yaml

deployment.apps "hostname" deleted

deployment.apps/hostname created


In [71]:
# 列出目前 deloyment
! kubectl get deploy

NAME       READY   UP-TO-DATE   AVAILABLE   AGE
hostname   0/1     1            0           0s


In [72]:
# 列出目前 pods
! kubectl get pods

NAME                        READY   STATUS              RESTARTS   AGE
hostname                    1/1     Running             0          13m
hostname-4df4s              1/1     Terminating         0          18s
hostname-5477bcd76c-bsgqs   1/1     Terminating         0          31m
hostname-5477bcd76c-k225l   1/1     Terminating         0          31m
hostname-5477bcd76c-mcz9p   1/1     Terminating         0          31m
hostname-85d476d6d4-z8rvf   0/1     ContainerCreating   0          0s
hostname-d9wkm              1/1     Terminating         0          18s
hostname-x8c2k              1/1     Terminating         0          18s


In [73]:
# 列出目前 pods
! kubectl get pods --show-labels

NAME                        READY   STATUS              RESTARTS   AGE   LABELS
hostname                    1/1     Running             0          13m   app=nginx,env=dev
hostname-4df4s              1/1     Terminating         0          19s   tier=frontend
hostname-5477bcd76c-bsgqs   1/1     Terminating         0          31m   app=hostname,pod-template-hash=5477bcd76c
hostname-5477bcd76c-k225l   1/1     Terminating         0          31m   app=hostname,pod-template-hash=5477bcd76c
hostname-5477bcd76c-mcz9p   1/1     Terminating         0          31m   app=hostname,pod-template-hash=5477bcd76c
hostname-85d476d6d4-z8rvf   0/1     ContainerCreating   0          1s    app=hostname,pod-template-hash=85d476d6d4
hostname-d9wkm              1/1     Terminating         0          19s   tier=frontend
hostname-x8c2k              1/1     Terminating         0          19s   tier=frontend


In [74]:
# 直列印pod的名稱
! kubectl get pods --selector=app=hostname \
  --no-headers \
  -o custom-columns=":metadata.name"

hostname-5477bcd76c-bsgqs
hostname-5477bcd76c-k225l
hostname-5477bcd76c-mcz9p
hostname-85d476d6d4-z8rvf


In [ ]:
# 嘗試把pod刪掉
# 看看deployment會怎麼處理
! kubectl delete pods $(kubectl get pods --selector=app=hostname --no-headers -o custom-columns=":metadata.name"); 
! echo;
! kubectl get pods --selector=app=hostname
! sleep 5 # wait 5 second 
! echo; echo after 5 seconds; echo;
! kubectl get pods --selector=app=hostname

pod "hostname-5477bcd76c-bsgqs" deleted
pod "hostname-5477bcd76c-k225l" deleted
pod "hostname-5477bcd76c-mcz9p" deleted
pod "hostname-85d476d6d4-z8rvf" deleted


In [ ]:
! kubectl scale --replicas=2 deployment hostname
! kubectl get deploy hostname
! echo;
! kubectl get pods --selector=app=hostname

In [ ]:
! kubectl scale deploy/hostname --replicas=4; echo;
! kubectl get pods --selector=app=hostname

In [ ]:
! kubectl get pods --selector=app=hostname

In [ ]:
! kubectl delete pods --selector=app=hostname; 
! echo; sleep 1;
! kubectl get pods --selector=app=hostname;

---

In [ ]:
! kubectl delete deployment.apps/hostname

In [ ]:
! kubectl delete deploy hostname

In [ ]:
! kubectl delete deploy/hostname